In [2]:
import requests
import selenium
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from IPython.display import display_html
import matplotlib.colors as mcolors


Chrome_driver_path = '/Users/xuhanling/Desktop/申請資料/work application/Yahoo fantasy/chromedriver' #指定到您放置chromedriver.exe的位置
web = 'https://login.yahoo.com/config/login?.src=fantasy&specId=usernameRegWithName&.intl=us&.lang=en-US&.done=https://basketball.fantasysports.yahoo.com/' #Yahoo Fantasy登入頁面
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('User-Agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36"') #建議可以點選F12看request header的項目
driver = webdriver.Chrome(service=Service(Chrome_driver_path), options=chrome_options)  #打開模擬的Chrome瀏覽器
driver.maximize_window() ###最大化視窗，因為我發現某些情況下，較小的視窗會導致往下移動的JS沒有辦法執行。

driver.get(web) ###開始進入登入頁面

# Wait for the username field to be clickable, then enter the username
elem_user = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.NAME, 'username'))
)
elem_user.clear()
elem_user.send_keys('sam5348911@yahoo.com.tw')
driver.find_element(By.ID, 'login-signin').click()
time.sleep(5)
# Wait for the password field to be clickable, then enter the password
elem_pass = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'login-passwd'))
)
elem_pass.send_keys('Sh5348911!')
driver.find_element(By.ID, 'login-signin').click()
time.sleep(5)

In [158]:
web2 = 'https://basketball.fantasysports.yahoo.com/nba/45199/matchup?week=10' #要爬取matchup分數的地方
driver.get(web2)

for i in range(0,10):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #到達網頁後要滾輪持續往下滑，才會出現matchup分數欄位
    time.sleep(1)
    
soup = BeautifulSoup(driver.page_source, "html.parser")  #把撈到的頁面丟給BeautifulSoup解析
pages = soup.find_all('div',class_='Ta-c Js-hidden') #找網址
names = soup.find_all('a',class_='F-link') #找隊名


In [163]:
###先搞定隊名
names2=[]
for n in names[29:41]:
    names2.append(n.text)

###搞定對戰連結
target_pages=[]
for p in pages:
    target_pages.append(p.a['href'])
    
target_pages2 = []
for p2 in target_pages:
    target_pages2.append('https://basketball.fantasysports.yahoo.com'+p2)


In [165]:
###撈取對戰分數，左半部與右半部各抓一次。
lg1=[]
lg2=[]

for p3 in target_pages2:
    driver.get(p3)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    score1 = soup.find_all('td',class_='Ta-c')
    time.sleep(5)
    
    for i in score1[0:16]:
        lg1.append(i.text)
    
    for i in score1[17:33]:
        lg2.append(i.text)

In [166]:
### 依序把抓到的欄位填入pandas中
# Calculate the step size for slicing based on the pattern in your example
step_size = 16
lg1_index = 0  # Initial index for lg1
lg2_index = 0  # Initial index for lg2
contest_index = 16  # Initial index for lg2

# Initialize an empty dictionary to collect the data
data = {}

# Loop through the list of team names
for i, name in enumerate(names2):
    # Check if the index is even or odd and slice from lg1 or lg2 accordingly
    if i % 2 == 0:  # if index is even, slice from lg1
        data[name] = lg1[lg1_index:lg1_index + contest_index]
        lg1_index += step_size
    else:  # if index is odd, slice from lg2
        data[name] = lg2[lg2_index:lg2_index + contest_index]
        lg2_index += step_size

In [167]:
# Create the DataFrame from the dictionary
df1 = pd.DataFrame(data)
df1['隊名'] =['FGM','FGA','FG%','FTM','FTA','FT%','3PTM','3PTA','3PT%','PTS','REB','AST','ST','BLK','TO','DD']
df1 = df1[['隊名','太陽雙槍','躺著看你哭',"Justin 's Idiot Team",'打爆作弊仔',"yu's First-Class Team",'維他命',"justin's Wonderful Team","阿勝's Cool Team",'hip怕man','小艸西', '硬頸跳水玩家', '阿淵']]


### 畫畫讓圖表數據更美觀
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
#matplotlib inlin

In [168]:
df2=df1
df2.iloc[0:,1:] = df2.iloc[0:,1:].astype('float')

In [169]:
df3 = df2.set_index('隊名')
df3 = df3.T

In [170]:
# List of columns you want to convert to numeric
columns_to_convert = ['FGM','FGA','FG%','FTM','FTA','FT%','3PTM','3PTA','3PT%','PTS','REB','AST','ST','BLK','TO','DD']

# Convert the specified columns to numeric
for column in columns_to_convert:
    df3[column] = pd.to_numeric(df3[column], errors='coerce')

In [171]:
# Apply the Styler
# Apply the provided styling logic to the dataframe
styled_df = df3.style.background_gradient(subset=[col for col in df3.columns if col != 'TO'], cmap='PiYG', axis=0)\
                  .background_gradient(subset=['TO'], cmap='PiYG_r', axis=0)
format_dict = {col: "{:.2f}" for col in columns_to_convert}

# Apply the formatting to the specified columns
styled_df = styled_df.format(format_dict)
# Set up your title or caption
title = "大學盟Week10"

# Apply the formatting and add a caption which acts like a title
styled_df = styled_df.format(format_dict).set_caption(title).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'black'),
        ('font-size', '16px'),
        ('text-align', 'center'),
        ('font-weight', 'bold')
    ]
}])
styled_df

隊名,FGM,FGA,FG%,FTM,FTA,FT%,3PTM,3PTA,3PT%,PTS,REB,AST,ST,BLK,TO,DD
太陽雙槍,222.00,457.00,0.49,98.00,120.00,0.82,73.00,194.00,0.38,615.00,185.00,125.00,28.00,23.00,57.00,6.00
躺著看你哭,184.00,403.00,0.46,80.00,103.00,0.78,46.00,168.00,0.27,494.00,139.00,96.00,16.00,11.00,51.00,3.00
Justin 's Idiot Team,230.00,459.00,0.50,118.00,138.00,0.85,64.00,154.00,0.42,642.00,193.00,160.00,26.00,20.00,57.00,6.00
打爆作弊仔,231.00,417.00,0.55,124.00,154.00,0.81,43.00,111.00,0.39,629.00,240.00,126.00,27.00,43.00,53.00,7.00
yu's First-Class Team,109.00,225.00,0.48,41.00,54.00,0.76,31.00,71.00,0.44,290.00,69.00,59.00,15.00,13.00,23.00,1.00
維他命,160.00,326.00,0.49,86.00,108.00,0.80,36.00,117.00,0.31,442.00,141.00,83.00,29.00,24.00,44.00,4.00
justin's Wonderful Team,232.00,438.00,0.53,113.00,155.00,0.73,67.00,167.00,0.40,644.00,182.00,172.00,35.00,18.00,73.00,10.00
阿勝's Cool Team,198.00,429.00,0.46,114.00,149.00,0.77,54.00,150.00,0.36,564.00,209.00,132.00,30.00,17.00,53.00,7.00
hip怕man,202.00,400.00,0.51,109.00,144.00,0.76,45.00,128.00,0.35,558.00,200.00,133.00,25.00,31.00,64.00,10.00
小艸西,229.00,453.00,0.51,152.00,189.00,0.80,35.00,107.00,0.33,645.00,224.00,139.00,18.00,19.00,69.00,10.00


In [164]:
names2

['太陽雙槍',
 '小艸西',
 "Justin 's Idiot Team",
 "yu's First-Class Team",
 '打爆作弊仔',
 'hip怕man',
 '維他命',
 '阿淵',
 "justin's Wonderful Team",
 '躺著看你哭',
 '硬頸跳水玩家',
 "阿勝's Cool Team"]

In [114]:
data

{'stat corrections': ['186',
  '379',
  '.491',
  '85',
  '109',
  '.780',
  '43',
  '117',
  '.368',
  '500',
  '214',
  '147',
  '30',
  '34',
  '63',
  '12'],
 '太陽雙槍': ['174',
  '357',
  '.487',
  '116',
  '139',
  '.835',
  '39',
  '110',
  '.355',
  '503',
  '192',
  '112',
  '27',
  '34',
  '62',
  '8'],
 '維他命': ['201',
  '381',
  '.528',
  '87',
  '106',
  '.821',
  '42',
  '109',
  '.385',
  '531',
  '201',
  '133',
  '33',
  '36',
  '48',
  '8'],
 "Justin 's Idiot Team": ['175',
  '368',
  '.476',
  '96',
  '118',
  '.814',
  '41',
  '107',
  '.383',
  '487',
  '176',
  '116',
  '22',
  '24',
  '50',
  '6'],
 '硬頸跳水玩家': ['208',
  '395',
  '.527',
  '112',
  '135',
  '.830',
  '29',
  '99',
  '.293',
  '557',
  '204',
  '126',
  '42',
  '27',
  '66',
  '11'],
 '打爆作弊仔': ['199',
  '429',
  '.464',
  '90',
  '109',
  '.826',
  '54',
  '147',
  '.367',
  '542',
  '156',
  '138',
  '29',
  '14',
  '54',
  '6'],
 "阿勝's Cool Team": ['180',
  '353',
  '.510',
  '67',
  '80',
  '.838',
 